Lecture 10 will cover exceptions, decorators

Reference
 * [2] Section 9.8-9.10

# Static methods

Static methods are methods that don't require the object itself.

Why are these needed?

Usually helper functions that "belong" to the class itself (i.e. are independent of actual instances)

In [1]:
from numpy.linalg import norm

class Car:
    america = True

    @staticmethod
    def kph2mph(speed):
        return speed / 1.60934

    def __init__(self, model, plate, vel):
        self.model = model
        self.velocity = vel

    def __str__(self):
        speed = norm(self.velocity)
        if self.america:
            speed = self.kph2mph(speed)
        return "Model:" + self.model + " going at " + str(speed)

In [2]:
car1 = Car("saab", "XZ123", 140)
print(car1)

Model:saab going at 86.99218313097295


In [3]:
Car.kph2mph(140)

86.99218313097295

# Decorators

We've seen two uses of decorators already:

```python
    @staticmethod
    def kph2mph(speed):
        return speed / 1.60934
        
    @abc.abstractmethod
    def calories(self):
        """ Returns the number of calories in the pizza """
```

In the broadest terms, decorators modify functionality. The code using `@`:
```python
@my_decorator
def my_function():
    return compute_stuff()
```
is basically the same as doing:
```python
def my_function():
    return compute_stuff()
my_function = my_decorator(my_function)
```

We can make a silly example of what it could do:

In [25]:
class always_print_hello:
    def __init__(self, f):
        self.f = f

    def __call__(self, x):
        print("Hello!")
        return self.f(x)

In [26]:
def my_function(x):
    return x+7

my_function = always_print_hello(my_function)
print(my_function(3))

Hello!
10


The usefulness of decorators are perhaps not immeditaly obvious. But with some ingenuity they can be used to enrich the language itself. For example, enforcing an IntEnum to be unique:

In [3]:
import enum
@enum.unique
class GameState2(enum.IntEnum):
    not_started = 0
    started = 1
    ended = 2
    paused = 2 # Opps

ValueError: duplicate values found in <enum 'GameState2'>: paused -> ended

You will not be required to write any decorators on the exam, though you should know about `@staticmethod` and `@abc.abstractmethod` and how these are used.
Abstract and static methods are re-occuring themes in object oriented languages, though they might be expressed differently (i.e. using different syntax) in different languages.

Other uses of decorators are more unique "language quirks", and excessive use just makes the code hard to understand, so don't go crazy with them!

# Error handling

When dealing with user input (or even programmer input, in the example above), errors are inevitable.
We can't always know if a given piece of code will always work (i.e. cause an error on certain user inputs)

The way to deal with this is to *try* to execute segments, and deal with the errors if they occur:

In [6]:
for x in range(2,-1,-1):
    try:
        print(1.0 / x)
    except:
        print("inf")

0.5
1.0
inf


Though, we probably want to check *what* error occured, and deal with each specifically

In [7]:
for x in range(2,-1,-1):
    try:
        print(1.0 / y) # Opps, wrong variable, not defined
    except:
        print("inf") # Not correct for this error

inf
inf
inf


In [8]:
for x in range(2,-1,-1):
    try:
        print(1.0 / x) # Opps, wrong variable, not defined
    except ZeroDivisionError:
        print("inf")

0.5
1.0
inf


In [13]:
stuff = [1,2,3,4]
for x in range(8,-1,-1):
    try:
        print(stuff[x])
    except IndexError:
        print("oor")
    except NameError:
        print("Wrong symbol!")

oor
oor
oor
oor
oor
4
3
2
1


This is why broadly catching all errors are very bad form. Don't do it!

### Using "as" to obtain info from errors

In [64]:
import sys

try:
    f = open('filename.txt')
    s = f.readline()
    i = int(s.strip())
except FileNotFoundError as e:
    print(e.errno)

2


Useful for cases where an exception might carry some useful information for how to proceed with the exception handling.
For example, when trying to remove a temporary file, we might get an IO error, but the error number might be indicating that the error was due to the file already being deleted (which we can just safely ignore), while some other error code (e.g. wrong permissions) might be more problematic.

### Finally and else

In [37]:
def f(x, y):
    return divide(x, y) * y

def divide(x, y):
    f = x/y
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        # Will run if we don't have an exception first:
        print("result is", result)
    finally:
        # This will always be executed last
        print("executing finally clause")

In [38]:
f(0,0)

ZeroDivisionError: division by zero

In [25]:
divide(2,3)

result is 0.6666666666666666
executing finally clause


# Making your own errors

An exception is just a class:

In [66]:
class MyCustomError(Exception):
    # Inheriting from the base class for all exceptions
    pass

class NegativeValueError(ValueError):
    # Subclassing an exception to add a useful(?) specific case
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return "Value is " + repr(self.value)

Catching the exception you can get any stored values like any object:

In [67]:
def my_function(x):
    if x < 0:
        raise NegativeValueError(x)
    return x * 3

In [68]:
x = -1
try:
    my_function(x)
except ValueError as e:
    print('Problem with x={}, try again.'.format(e.value))

Problem with x=-1, try again.


Uncaught exceptions prints the string representations of the class:

In [69]:
my_function(-2)

NegativeValueError: Value is -2

## Possiblecustom exceptions

** Question **
* In your Poker-game library, what would be some suitable exceptions to implement?

---------------------

---------------------

---------------------
* MissingCardError
* OutOfMoneyError
* EmptyDeckError

# Question 4 (7p total)

In this question, you will implement classes for representing a simple file structure, which would be useful when
working on a library for a archive (like a zip file).
For the sake of keeping the exam question reasonably short, we will not consider the actual file contents, but only the
metadata (file structure, file names etc.)

You should generate some XML markup as shown in the example. (XML is simple markup language for storing and transporting
data.)

**Example**
```python
some_files = [File('Important data.dat', True), File('"Quotes" & jokes.txt', False)]
some_code = [File('important_python_code.py', False), File('cheat_sheet.py', False)]
code_dir = Directory('Code', some_code)
root = Directory('Directories can also have \'escaped\' characters', some_files)
root.add_node(code_dir)
print(root.xml_structure())
print("Total number of nodes in the tree is {}".format(len(root)))
```
should yield

```xml
<directory name="Directories can also have &apos;escaped&apos; characters">
    <file name="Important data.dat" binary="yes" />
    <file name="&quot;Quotes&quot; &amp; jokes.txt" binary="no" />
    <directory name="Code">
        <file name="important_python_code.py" binary="no" />
        <file name="cheat_sheet.py" binary="no" />
    </directory>
</directory>
```
```text
Total number of nodes in the tree is 6
```

## Part A (2.5p)
Create a base class `Node` stores a variable `name`.
This class should have:

- a method `escaped_name` that returns the name of the node but every with special characters replaced (see list below).
- an abstract method `xml_structure` that takes an optional argument `indent` with a default value of 0
- an abstract operator for `len()` that computes the number of nodes in the entire tree recursively.

*Note: the name "node" is typically used in file systems for referring to an object, like a directory, file or link.*

- ampersand (`&`) is escaped to `&amp;`
- double quotes (`"`) are escaped to `&quot;`
- single quotes (`'`) are escaped to `&apos;`
- less than (`<`) is escaped to `&lt;`
- greater than (`>`) is escaped to `&gt;`

In [48]:
import abc

class Node(metaclass=abc.ABCMeta):
    escapes = [('&', '&amp;'),
               ('"', '&quot;'),
               ("'", '&apos;'),
               ('>', '&gt;'),
               ('<', '&lt;')]
    
    def __init__(self, name):
        self.name = name
        
    def escaped_name(self):
        esc_name = self.name
        for orig, esc in self.escapes:
            esc_name = esc_name.replace(orig, esc)
        return esc_name
    
    @abc.abstractmethod
    def __len__(self):
        pass

    @abc.abstractmethod
    def xml_structure(self, indent=0):
        pass

## Part B (2.5p)
Create a subclass `Directory` that takes a `name` and a list of `nodes` that it should contain:

- a method `xml_structure(indent)` should return a string that starts with 
`<directory name="escaped name here">`
then lists xml structure all the contained nodes, then ends with
`</directory>`
The contained nodes should be indented 4 spaces more than the current indent. See the example below on how it should look.

- a method `add_node(node)` that adds a node to the directory.
- Using `len` on a `Directory` object should return the sum of the `len` of all the contained nodes + 1 (for the directory itself).

In [57]:
class Directory(Node):
    def __init__(self, name, nodes):
        super().__init__(name)
        self.nodes = nodes
    
    def add_node(self, node):
        self.nodes.append(node)
        
    def __len__(self):
        s = 1
        for node in self.nodes:
            s += len(node)
        return s
    
    def xml_structure(self, indent=0):
        xml = (' ' * indent) + '<directory name="{}">\n'.format(self.escaped_name())
        for node in self.nodes:
            xml += node.xml_structure(indent+4)
        xml += (' ' * indent) + '</directory>'
        return xml

## Part C (2p)
Implement a subclass `File` that takes a `name` and a bool `binary` which indicates if the file contains
binary data (or just text):

- The `len` of a file is always 1.
- Implement a method `xml_structure(indent)` that should return a string that looks like:
```xml
<file name="escaped name here" binary="yes or no" />
```
The string should be preceded with as many spaces as indicated by `indent`. Remember a line break (see the example)!

In [62]:
class File(Node):
    def __init__(self, name, binary):
        super().__init__(name)
        self.binary = binary
    
    def __len__(self):
        return 1
    
    def xml_structure(self, indent=0):
        return (' ' * indent) + '<file name="{}" binary="{}" />\n'.format(
                self.escaped_name(), "yes" if self.binary else "no")

In [63]:
# Test code (as provided for exam)
some_files = [File('Important data.dat', True), File('"Quotes" & jokes.txt', False)]
some_code = [File('important_python_code.py', False), File('cheat_sheet.py', False)]
code_dir = Directory('Code', some_code)
root = Directory('Directories can also have "escaped" characters', some_files)
root.add_node(code_dir)
print(root.xml_structure())
print("Total number of nodes in tree is {}".format(len(root)))

<directory name="Directories can also have &quot;escaped&quot; characters">
    <file name="Important data.dat" binary="yes" />
    <file name="&quot;Quotes&quot; &amp; jokes.txt" binary="no" />
    <directory name="Code">
        <file name="important_python_code.py" binary="no" />
        <file name="cheat_sheet.py" binary="no" />
    </directory></directory>
Total number of nodes in tree is 6
